In [1]:
import polars as pl

In [7]:
header_columns = """CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
""".split(",")
for i, c in enumerate(header_columns):
    print(i+1, c)

1 CMTE_ID
2 AMNDT_IND
3 RPT_TP
4 TRANSACTION_PGI
5 IMAGE_NUM
6 TRANSACTION_TP
7 ENTITY_TP
8 NAME
9 CITY
10 STATE
11 ZIP_CODE
12 EMPLOYER
13 OCCUPATION
14 TRANSACTION_DT
15 TRANSACTION_AMT
16 OTHER_ID
17 TRAN_ID
18 FILE_NUM
19 MEMO_CD
20 MEMO_TEXT
21 SUB_ID



In [5]:
import geopandas as gpd
li_zip_codes = gpd.read_file("../references/long_island_zipcodes.geojson")
zip_codes = li_zip_codes.postalcode.unique().tolist()

Bad Data! We have someone in there as 
- zipcode 'W11 3LN'. As it turns out, this is a postal code in England (!!!!!)
- tranaction_dt 'LINCOLN' (!)
- shape error, because some of the data is so badly formed that it is missing entire columns

That last one is fatal to pretty much any attempt to read data. It means that we would have to do extra work, testing each record to make sure it is actually complete. 

the data is in not great shape, unfortunately. That's always the case with things like this. 


This scanned 18GB of data from my machine in just over a minute. 

In [8]:
fec_df = pl.scan_csv(
    "../data/raw/by_date/*.txt", 
    separator='|', 
    has_header=False, 
    infer_schema_length=0,
    ignore_errors=True,
    missing_utf8_is_empty_string=True,
).filter(pl.col("column_10") == "NY").filter(pl.col("column_11").str.slice(0,5).is_in(zip_codes)).collect(streaming=True)

STREAMING CHUNK SIZE: 2380 rows


Now that we have a subset of the data, we can apply our header column to it so we can get the names we want to work with, which were taken from here:

https://www.fec.gov/campaign-finance-data/contributions-individuals-file-description/

In [11]:
fec_df.columns = header_columns

We can then write the file to parquet, which will let us use it with Pandas in our next notebook. 

In [12]:
fec_df.write_parquet("../data/processed/individual_2020_li.parquet")